In [5]:
import ROOT
from ROOT import gStyle
import numpy as np

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list,first_binning):
    for hist in hist_list:
        for i in range(0,len(rebining)):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=1
            if hist.GetBinWidth(i)!=first_binning:
                sf=hist.GetBinWidth(i)/first_binning
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)
            
ztt_sample="Ztautau_Sherpa.root"

In [6]:
histos_no_rebin={
"delta_phi":[],
"delta_y":[],
"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],
"omega":[],
}
histos_rebin={
"rnn_score_1p":[0.8,0.04,0.04],
"rnn_score_3p":[0.8,0.04,0.04],
"lep_pt":[120,5,10],
"tau_pt":[120,5,10],
"ljet0_pt":[120,5,10],
"ljet1_pt":[120,5,10],
"pt_bal":[0.3,0.02,0.02],
"mass_jj":[1000,50,200],
"Z_centrality":[2,0.05,1],
"leptau_mass":[120,5,10],
"eBDT":[0.3,0.05,0.05],
"reco_mass_i":[120,5,10],
"reco_mass_o":[120,5,10],
}
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
for i in histos:
    print(i)
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Ztautau=ROOT.TFile.Open(ztt_sample,"READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Zjets.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")
    Signal=ROOT.TFile.Open("Signal.root","READ")

    gStyle.SetOptStat(1111111)
    a=i

    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    ztt=Ztautau.Get(a)

    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    ztt.SetDirectory(0)
    
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())

    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    Ztautau.Close()

    hs = ROOT.THStack("hs","")
    
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(s,histos[i][0],e,histos[i][1],histos[i][2])
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"signal",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        ztt=ztt.Rebin(nb,"ztt",rebining)
        
        if histos[i][1]!=histos[i][2]:
            hist_list=[signal,data,vv,tt,t,z,w,ztt]
            normalization(hist_list,histos[i][1])

    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    data.SetLineColor(ROOT.kBlack)
    ztt.SetLineColor(ROOT.kViolet)
    ztt.SetFillColor(ROOT.kViolet)



    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(signal)
    hs.Add(ztt)
    

    ratio=mc.Clone()
    ratio.Divide(data)

    gStyle.SetOptStat(1111111)

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)

    pad1.Draw ()
    pad1.cd ()




    data.Draw("pe")
    hs.Draw("HIST same")
    data.Draw("pe same")
    #if "rnn_score" in i:
    pad1.SetLogy()

    
    data . GetYaxis (). SetRangeUser (1 ,1.2*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s ,e)
    data . GetYaxis (). SetTitle ("Events/5 GeV")

    hs . GetXaxis (). SetRangeUser (s ,e)
    legend = ROOT . TLegend (0.35 ,0.15 ,0.45 ,0.35)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( ztt ," Ztt")

    legend . SetLineWidth (0)
    legend . Draw ()



    hs.SetTitle(a)

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.05 ,1 ,0.38)
    pad2.Draw ()
    pad2.cd ()
    ratio.Draw ("pe")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (0 ,2.0)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line . Draw (" same ")


    canvas3.Print(a+".pdf")
    
    if a=="tau_nprongs_cuts_tpt":
        mc_yield_1p=mc.Integral(1,2)
        data_yield_1p=data.Integral(1,2)
        print("\n"*2)
        print("MC 1p= ",round(mc_yield_1p,3))
        print("DATA 1p= ",round(data_yield_1p,3))
        print("\n"*2)
        mc_yield_3p=mc.Integral(3,4)
        data_yield_3p=data.Integral(3,4)
        print("\n"*2)
        print("MC 3p= ",round(mc_yield_3p,3))
        print("DATA 3p= ",round(data_yield_3p,3))
        print("\n"*2)


delta_phi
delta_y
n_bjets
lepiso
n_jets_interval
omega
rnn_score_1p
rnn_score_3p
lep_pt
tau_pt
ljet0_pt
ljet1_pt
pt_bal
mass_jj
Z_centrality
leptau_mass
eBDT
reco_mass_i
reco_mass_o


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_phi.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_y.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_bjets.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lepiso.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_jets_interval.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file omega.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file rnn_score_1p.pdf has been created
Warning in <TCa

In [10]:
data.Integral(11,20)
data.GetXaxis().GetBinUpEdge(data.GetNbinsX())

200.0

### PLOT SINGULAR PROPERTIES

In [ ]:

File="Wjets.root"
histo="tau_matched"

Signal=ROOT.TFile.Open(File,"READ")
signal=Signal.Get(histo)
canvas4=ROOT.TCanvas("canvas4")
canvas4.cd()
#signal=signal.Rebin(50,"signal",rebining)
signal.Draw("HIST")
signal.SetFillColor(ROOT.kGreen)
signal.SetLineColor(ROOT.kGreen)
signal.SetTitle(histo)

legend = ROOT.TLegend (0.5 ,0.5 ,0.85 ,0.55)
legend.AddEntry ( signal ,File[:-5])
legend.Draw("same")

canvas4.Print(File[:-5]+"_"+histo+".pdf")

### PLOTTING BDT SCORE

In [ ]:
canvas5=ROOT.TCanvas("canvas5")
canvas5.cd()
w.Draw("HIST")
z.Draw("HIST,same")
vv.Draw("HIST,same")
tt.Draw("HIST,same")
t.Draw("HIST,same")
signal.Draw("HIST,same")


w.SetLineColor(ROOT.kGreen)

z.SetLineColor(ROOT.kRed)

vv.SetLineColor(ROOT.kBlue)

tt.SetLineColor(ROOT.kYellow)

t.SetLineColor(ROOT.kCyan)

legend.Draw()

canvas5.SetLogy()

canvas5.Print("BDT_score.pdf")



In [ ]:
rebining

In [ ]:
MC=ROOT.TFile.Open("MC.root","READ")
Signal=ROOT.TFile.Open("Signal.root","READ")
VV=ROOT.TFile.Open("VV.root","READ")
Wjets=ROOT.TFile.Open("Wjets.root","READ")
Zjets=ROOT.TFile.Open("Zjet.root","READ")
Ttbar=ROOT.TFile.Open("ttbar.root","READ")
SingleTop=ROOT.TFile.Open("singletop.root","READ")

gStyle.SetOptStat(1111111)
a="reco_mass_jetn_btag_iso_bdt_tmass_omega"

mc=MC.Get(a)
signal=Signal.Get(a)
vv=VV.Get(a)
w=Wjets.Get(a)
z=Zjets.Get(a)
tt=Ttbar.Get(a)
t=SingleTop.Get(a)

mc.SetDirectory(0)
signal.SetDirectory(0)
vv.SetDirectory(0)
w.SetDirectory(0)
z.SetDirectory(0)
tt.SetDirectory(0)
t.SetDirectory(0)

MC.Close()
Signal.Close()
VV.Close()
Wjets.Close()
Zjets.Close()
Ttbar.Close()
SingleTop.Close()

hs = ROOT.THStack("hs","")
#'''
ob=300
s=0
e=300
n=6
nb=int(ob/n)
rebining=np.arange(s,e+n*(e-s)/ob,n*(e-s)/ob)
mc=mc.Rebin(nb,"signal",rebining)
signal=signal.Rebin(nb,"signal",rebining)
#data=data.Rebin(nb,"data",rebining)
vv=vv.Rebin(nb,"vv",rebining)
tt=tt.Rebin(nb,"tt",rebining)
t=t.Rebin(nb,"t",rebining)
z=z.Rebin(nb,"z",rebining)
w=w.Rebin(nb,"w",rebining)
#'''

w.SetFillColor(ROOT.kGreen)
w.SetLineColor(ROOT.kGreen)
z.SetFillColor(ROOT.kRed)
z.SetLineColor(ROOT.kRed)
vv.SetFillColor(ROOT.kBlue)
vv.SetLineColor(ROOT.kBlue)
tt.SetFillColor(ROOT.kYellow)
tt.SetLineColor(ROOT.kYellow)
t.SetFillColor(ROOT.kCyan)
t.SetLineColor(ROOT.kCyan)




hs.Add(vv)
hs.Add(w)
hs.Add(z)
hs.Add(tt)
hs.Add(t)
hs.Add(signal)




canvas6 = ROOT.TCanvas("canvas8")
canvas6.cd()


hs.Draw("HIST")

legend = ROOT . TLegend (0.8 ,0.4 ,0.9 ,0.6)
legend . AddEntry ( vv ,"Diboson")
legend . AddEntry ( signal ," Signal ")
legend . AddEntry ( w ," W+jets ")
legend . AddEntry ( z ," Z+jets ")
legend . AddEntry ( t ," Single top ")
legend . AddEntry ( tt ," ttbar ")

legend . SetLineWidth (0)
legend . Draw ()



hs.SetTitle(a)





canvas6.Print(a+".pdf")
